# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow

tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-macos==2.10.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)

    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)

        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]

        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]

    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2024-08-15 12:37:39.046023: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "INVALID_ARGUMENT: Error executing an HTTP request: HTTP response code 400 with body '{"error":"invalid_grant","error_description":"Invalid grant: account not found"}'". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFQX4HX/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFQX4HX/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2024-08-15 12:38:02.664187: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [97]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [18]:
wv['dinosaur']

array([-0.03677012,  0.06672348,  0.01578667,  0.07454763, -0.00028925,
       -0.11310486,  0.00640256,  0.1901727 , -0.09795486, -0.04844794,
       -0.02800705, -0.12249455,  0.065918  ,  0.0461825 ,  0.03663136,
       -0.10284461,  0.07790668, -0.08793945, -0.01883735, -0.1627195 ,
        0.07276753,  0.03354257,  0.1109184 , -0.07642487, -0.00151322,
       -0.01902177, -0.11611068, -0.01157949, -0.02981343, -0.03593576,
        0.08402731, -0.01404711,  0.06974266, -0.06376919, -0.0690884 ,
        0.07146487,  0.03511704, -0.07184157, -0.068377  , -0.15639706,
       -0.02530811, -0.12024137, -0.01604012,  0.03049854,  0.08907567,
       -0.00876094, -0.03854279, -0.07944999,  0.01019354,  0.06780097,
        0.05271006,  0.00242727, -0.10501487, -0.01108396, -0.05897209,
        0.04371815,  0.07326361,  0.04620436, -0.07709966, -0.01810523,
       -0.01155083,  0.00027483, -0.00773416,  0.05267601, -0.05405983,
        0.10206532,  0.01987046,  0.00949169, -0.10281659,  0.06

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [27]:
from gensim.models.word2vec.wv import most_similar

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

In [21]:
rep_size = len(wv['dog'])

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [39]:
similar_words = word2vec.wv.most_similar('dog')

print(similar_words)

[('green', 0.9898310303688049), ('anna', 0.9885790944099426), ('pot', 0.9868289828300476), ('pole', 0.9847841858863831), ('vice', 0.9831891059875488), ('fortune', 0.9826832413673401), ('attitude', 0.9811385273933411), ('suicide', 0.9803786873817444), ("girl's", 0.9800941348075867), ('bat', 0.9798561334609985)]


📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [42]:
similar_words_vector = word2vec.wv.similar_by_vector('dog')

print(similar_words_vector)

[('green', 0.9898310303688049), ('anna', 0.9885790944099426), ('pot', 0.9868289828300476), ('pole', 0.9847841858863831), ('vice', 0.9831891059875488), ('fortune', 0.9826832413673401), ('attitude', 0.9811385273933411), ('suicide', 0.9803786873817444), ("girl's", 0.9800941348075867), ('bat', 0.9798561334609985)]


# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [48]:
wv['good'] - wv['bad']

array([-3.82213593e-01, -2.91648924e-01,  2.31884599e-01,  4.42175388e-01,
        1.07193708e-01, -2.36256957e-01, -2.08967656e-01,  2.32231095e-02,
       -2.19263196e-01, -3.28810811e-01,  1.74571261e-01, -2.53212333e-01,
        1.68228149e-01,  7.39210546e-02, -9.94296372e-02,  4.21591222e-01,
       -3.66613269e-01,  3.82568724e-02, -4.36322212e-01, -1.20495915e-01,
       -1.81153417e-03,  1.62798405e-01,  1.89263999e-01,  7.52955675e-04,
        1.06712878e-02, -1.39503226e-01, -1.08507186e-01,  7.66563833e-01,
       -1.78951442e-01, -6.80743217e-01, -2.90498853e-01,  2.75481492e-01,
        1.26160562e-01,  2.50371128e-01,  4.38057184e-02, -5.86020947e-01,
        2.95028120e-01,  4.44483161e-02, -3.11272681e-01,  9.93721247e-01,
       -2.18903452e-01,  2.69331932e-02, -5.12723744e-01, -3.60594153e-01,
        2.74874926e-01, -1.35819539e-01,  7.13044524e-01,  8.41081142e-03,
       -1.81720942e-01,  4.31292236e-01, -1.90505460e-01,  3.43875885e-02,
       -4.67521876e-01,  

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [50]:
res = wv['good'] - wv['bad'] + wv['stupid']

print(res)

[-0.04184988 -0.35822192  0.31309354  0.07148799  0.18175331 -0.7674772
 -0.30802587  0.40380377 -0.55408543 -0.5981736   0.01673548 -0.5159663
  0.17161293  0.31767794  0.09470835  0.39940315 -0.10785487 -0.22321792
 -0.55883765 -0.78765893  0.3359533   0.265031    0.54011387 -0.06729041
 -0.1516144  -0.24606276 -0.2836427   0.60182726 -0.45606136 -0.66234946
  0.35930794  0.38617238  0.36044174 -0.06373683 -0.38998538  0.16115028
  0.30635366  0.09064821 -0.6150015   0.00535357 -0.20239036 -0.5220874
 -0.35423562  0.12255368  0.5400102  -0.01890925  0.27932763 -0.5143149
 -0.13874528  0.61745054 -0.1169391  -0.26277232 -0.64289063  0.2486263
  0.23234452  0.36113405  0.06069373  0.17950907 -0.35655913  0.483551
  0.1986866  -0.02289507  0.16620117 -0.04122305 -0.5188458   0.5363205
 -0.5261341  -0.10570097  0.06190643  0.66297996 -0.58639306 -0.13496351
  0.50701874  0.2706775   0.6149737   0.0615117   0.01128703 -0.30522496
 -0.08296427  0.14435992  0.18933275 -0.15570481 -0.6211929

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [53]:
closest_res = word2vec.wv.similar_by_vector(res)

print(closest_res)

[('nice', 0.750499963760376), ('potential', 0.7326016426086426), ('good', 0.7248304486274719), ('given', 0.7247084975242615), ('great', 0.7155494689941406), ('always', 0.7148999571800232), ('also', 0.7091274857521057), ('backbone', 0.70650315284729), ('tough', 0.7032467722892761), ('incredible', 0.7026028037071228)]


Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [56]:
boy_girl = wv['boy'] - wv['girl']

print(boy_girl)

[ 0.1224682   0.08271772 -0.04362607  0.03449422 -0.07577558 -0.16082951
  0.06026831  0.0400151  -0.03233565  0.10518828  0.13133107 -0.01075563
  0.08913681  0.1551781   0.10420342 -0.00840783  0.01401822 -0.13505495
 -0.09585131 -0.32093513  0.20534076  0.05088255  0.2107051  -0.09225562
 -0.10430351 -0.01396348  0.03532672 -0.29019898 -0.00806633 -0.04252311
  0.18312192  0.17211586  0.03929485  0.18909806 -0.05983203  0.02672325
 -0.00199555  0.0327993  -0.13927573 -0.24822661 -0.03069869  0.09805404
  0.19018668 -0.23315683  0.09514043  0.12820315 -0.00606991  0.04305117
 -0.08087057  0.17662406 -0.07925472  0.08317569 -0.2234326   0.00967948
 -0.07569559 -0.09665605  0.06310375  0.09390031 -0.08561765 -0.00479233
  0.00718227 -0.13918604 -0.07115898 -0.05267457 -0.11498317 -0.03760853
 -0.22755787 -0.26209944 -0.02162468 -0.08279842 -0.00802857 -0.0307117
 -0.26829273  0.02863342 -0.32944256 -0.22639126  0.14897746  0.00983001
 -0.08186142  0.01625034  0.3923843  -0.02020589  0.

In [57]:
king_queen = wv['king'] - wv['queen']

print(king_queen)

[ 0.23772103  0.05564313 -0.1262919  -0.0580668   0.01011919 -0.07867801
 -0.07547373  0.0624423  -0.01955056 -0.14206657  0.08405781  0.05705294
  0.00349248  0.07698509  0.07682502  0.07265113  0.06355548  0.05752857
  0.00533827 -0.12937272  0.10369599 -0.09929205  0.23056075 -0.15585823
  0.00104243 -0.01256614  0.00052544 -0.01763994 -0.11633019 -0.02861921
  0.01533948  0.02525093  0.04759042 -0.08215114 -0.08190696  0.05971275
 -0.00309344  0.07015042 -0.09077688 -0.11047016 -0.01263449 -0.1793636
 -0.06306213  0.11276232  0.02590477  0.05842102 -0.19577904 -0.15649194
  0.0936272   0.17907642 -0.00055425 -0.14032453 -0.26203638  0.1155656
  0.07387322  0.05671096  0.10177906 -0.02312963 -0.08345098 -0.13561392
  0.02798004 -0.15216209  0.06323096  0.09675372 -0.03651364  0.27829453
 -0.06334027 -0.03505634 -0.10647732 -0.05765107 -0.07034293  0.09983629
 -0.00122204  0.03562951  0.16852593 -0.08519375  0.02795892  0.02070606
 -0.04907624 -0.04654772 -0.02901515  0.04059915 -0.0

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [58]:
word2vec_2 = Word2Vec(sentences=X_train,vector_size=200)

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [59]:
wv_2 = word2vec_2.wv

In [62]:
wv_2['dinosaur']

array([ 3.77861001e-02,  1.05817681e-02,  9.65510868e-03,  1.81067605e-02,
        8.75672027e-02, -5.93355149e-02,  2.89230980e-02,  1.32578596e-01,
       -3.63099165e-02,  3.15793753e-02,  4.61568270e-05, -4.22111899e-02,
       -1.26072811e-03,  3.79950851e-02, -2.25219652e-02, -1.71439145e-02,
       -1.32807810e-03,  7.05896737e-03,  2.56578773e-02, -1.37417629e-01,
        5.95845357e-02, -5.66588081e-02,  2.05704216e-02, -2.21678466e-02,
       -3.11432201e-02, -3.27340364e-02, -3.01245879e-02, -2.37270240e-02,
       -3.52942199e-02,  2.99172550e-02,  3.78471762e-02,  1.54138375e-02,
        4.29157428e-02,  2.21374235e-03,  2.00664569e-02,  5.15169464e-02,
        3.24439295e-02, -3.16959247e-02, -2.12813597e-02, -6.50635064e-02,
       -2.68295798e-02, -3.42815928e-03, -3.63743049e-03,  3.03195184e-03,
        4.84209694e-02, -1.40306652e-02, -1.47853894e-02, -2.61673741e-02,
        2.79976483e-02,  3.93726937e-02, -6.86238566e-03, -2.12649014e-02,
       -3.17898281e-02, -

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [67]:
word2vec_3 = Word2Vec(sentences=X_train,vector_size=200,min_count=10)

In [68]:
wv_3 = word2vec_3.wv

In [69]:
wv_3['dog']

array([ 0.06773046, -0.0286838 , -0.05136425,  0.17785399,  0.20817798,
       -0.12602589,  0.09093608,  0.31276414, -0.02541702,  0.11449323,
       -0.02136367, -0.15385872, -0.0544573 ,  0.02406974, -0.14668174,
       -0.11324307, -0.06614311, -0.00064602,  0.01862523, -0.45383102,
        0.10747163, -0.16120453,  0.01226396,  0.06781401, -0.0150787 ,
       -0.14557834, -0.05899752, -0.19738105, -0.17983422,  0.0193023 ,
        0.09569991, -0.01614401,  0.13880678, -0.06591661,  0.02702117,
        0.1351773 ,  0.14582229, -0.04050424, -0.12246624, -0.29042017,
       -0.12403852, -0.00630615,  0.00468972,  0.13189387,  0.28957748,
        0.00363773, -0.11149569, -0.09285882,  0.17845222,  0.07203821,
        0.00651459,  0.01654142, -0.06733306, -0.22398163,  0.01548517,
       -0.05224536,  0.07810136, -0.20001866, -0.12531467, -0.084594  ,
       -0.05043268,  0.04427451,  0.07685544, -0.0937945 , -0.30409783,
        0.07193495,  0.00701868,  0.36373273, -0.18036667,  0.24

Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [70]:
word2vec_2 = Word2Vec(sentences=X_train,vector_size=200,min_count=10, window= 10)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [79]:
from tensorflow.keras.layers import Normalization, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [101]:
word2vec.wv['dog']

array([-0.19977482,  0.16176112, -0.2133615 ,  0.26120007, -0.0364569 ,
       -0.38109854,  0.00672944,  0.6607357 , -0.1982248 , -0.21254796,
       -0.02201382, -0.3318978 , -0.06357048,  0.1724018 , -0.04827213,
       -0.24699375,  0.15830696, -0.271666  ,  0.01917258, -0.25320178,
        0.16961886,  0.08094141,  0.258622  , -0.10274209,  0.00868954,
        0.02226807, -0.2255721 , -0.06030077, -0.1499503 ,  0.0156923 ,
        0.17466827, -0.01916505,  0.13236949, -0.33285487, -0.13386829,
        0.17857867,  0.13887239, -0.15930764, -0.20695807, -0.24892683,
       -0.16223875, -0.32143003, -0.28293306,  0.16317527,  0.3338764 ,
       -0.0224509 , -0.18751782, -0.1342226 ,  0.2690664 ,  0.19649309,
        0.06930861, -0.24118967, -0.26605496, -0.04675134, -0.23480389,
        0.2323412 ,  0.18567751, -0.08111572, -0.2134571 ,  0.02954181,
        0.10312027, -0.0036335 ,  0.0060729 ,  0.01094742, -0.22360994,
        0.17969279, -0.04118614,  0.17428441, -0.28821132,  0.27

In [103]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return embedded_sentence

In [108]:

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']


def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])

    return np.array(embedded_sentence)

### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [ ]:
sentences = [example, example_missing_words]

In [115]:
def embedding(word2vec, sentences):
    embedded_sentences = []
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec,sentence)
        embedded_sentences.append(embedded_sentence)

    return embedded_sentences


X_train_embedded = embedding(word2vec, X_train)
X_test_embedded = embedding(word2vec, X_test)

In [117]:
len(X_train_embedded)

2500

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [118]:
X_train_pad = pad_sequences(X_train_embedded, dtype='float32', padding='pre')
X_test_pad = pad_sequences(X_test_embedded, dtype='float32', padding='pre')

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
